## Break into functions:

- Loading price + adding skip logic
    - Add columns
    - Get price
    - Generate prices
- Analyze position
    - Roll
    - Reduce size
- Generate orders
- Generate option data

In [11]:
%load_ext autoreload
%autoreload 2
from algo.positions.dataclasses import EODPositionData, PortfolioPositions, ActivePosition
from algo.positions.loaders.option_data import load_positions_data, get_orders_table
from algo.positions.loaders.snapshot_utils import build_snapshot_table
from algo.strategies.init_config import load_config
from EventDriven.riskmanager.actions import ROLL, EXERCISE, ADJUST, HOLD, RMAction
from dataclasses import dataclass, field
from algo.strategies.init_run import init_produce_orders, produce_orders, produce_signals
from datetime import datetime
from trade.helpers.helper import ny_now
from EventDriven.backtest import OptionSignalBacktest
import pandas as pd
from dbase.database.SQLHelpers import DatabaseAdapter, dynamic_batch_update, get_engine
import numpy as np
db = DatabaseAdapter()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Saving limits after backtest

In [12]:

def create_limits_table(bkt: OptionSignalBacktest, date: datetime = None):
    data = []
    for info, limits in bkt.risk_manager.limits_meta.items():
        for l in limits:
            if date is not None:
                if pd.to_datetime(info[2]).date() != date.date():
                    continue
            
            log = dict(
                signal_id=info[0],
                trade_id=info[1],
                date=info[2],
                strategy_name='long_bbands',
                risk_measure=l,
                value = limits[l],
            )
            data.append(log)


    df = pd.DataFrame(data)
    return df


In [13]:
from sqlalchemy import text

MEASURES = ('delta','gamma','vega','theta')
from sqlalchemy import text

MEASURES = ('delta','gamma','vega','theta')

def update_greeks_case(trade_id: str,
                       strategy_name: str,
                       signal_id: str,
                       greeks: dict,
                       date_):
    # validate inputs
    missing = [k for k in MEASURES if k not in greeks]
    if missing:
        raise ValueError(f"Missing greek(s): {missing}")

    # Support either greeks["new_date"] or greeks["date"] as the target date
    new_date = greeks.pop("new_date", greeks.pop("date", None))

    params = {
        "old_date": date_,                    # date you are updating FROM
        "signal_id": signal_id,
        "strategy_name": strategy_name,
        "trade_id": trade_id,
        **greeks
    }
    set_date_sql = ", `date` = :new_date" if new_date is not None else ""
    if new_date is not None:
        params["new_date"] = new_date

    # Update values (+ optionally move to new date)
    q = text(f"""
        UPDATE `limits`
        SET `value` = CASE `risk_measure`
            WHEN 'delta' THEN :delta
            WHEN 'gamma' THEN :gamma
            WHEN 'vega'  THEN :vega
            WHEN 'theta' THEN :theta
            ELSE `value`
        END
        {set_date_sql}
        WHERE `date`=:old_date
          AND `signal_id`=:signal_id
          AND `strategy_name`=:strategy_name
          AND `trade_id`=:trade_id
          AND `risk_measure` IN ('delta','gamma','vega','theta')
    """)

    engine = get_engine("portfolio_data")
    with engine.begin() as conn:
        res = conn.execute(q, params)

        # Verify at the final date (new_date if provided, else old_date)
        verify_date = new_date or date_
        rows = conn.execute(text("""
            SELECT `risk_measure`, `value`
            FROM `limits`
            WHERE `date`=:date AND `signal_id`=:signal_id
              AND `strategy_name`=:strategy_name AND `trade_id`=:trade_id
              AND `risk_measure` IN ('delta','gamma','vega','theta')
        """), {
            "date": verify_date,
            "signal_id": signal_id,
            "strategy_name": strategy_name,
            "trade_id": trade_id
        }).all()

    after = {rm: v for rm, v in rows}
    return {
        "matched": len(after) == 4,           # how many of the 4 measures now exist at the target date
        "rowcount": res.rowcount,             # may be 0 if values were identical
        "date": verify_date,
        "after": after
    }


In [14]:
def save_limits_by_meta(trade_id: str,
                strategy_name: str,
                signal_id: str,
                greeks: dict,
                date_):
    # validate inputs
    missing = [k for k in MEASURES if k not in greeks]
    if missing:
        raise ValueError(f"Missing greek(s): {missing}")
    params = {
        "date": date_,
        "signal_id": signal_id,
        "strategy_name": strategy_name,
        "trade_id": trade_id,
        **greeks
    }
    insert_rows = []
    for measure in MEASURES:
        insert_rows.append({
            "date": params["date"],
            "signal_id": params["signal_id"],
            "strategy_name": params["strategy_name"],
            "trade_id": params["trade_id"],
            "risk_measure": measure,
            "value": params[measure]
        })
    db = DatabaseAdapter()
    db.save_to_database(
        db='portfolio_data',
        table_name='limits',
        data=pd.DataFrame(insert_rows),
        filter_data=False
    )

In [15]:

def store_position_limits(delta_limit:str,
                                 gamma_limit:str,
                                 vega_limit:str,
                                 theta_limit:str,
                                 trade_id:str,
                                 strategy_name:str,
                                 date:str|datetime,
                                 signal_id:str):
    """"""

    ## First check if the limits exist
    if isinstance(date, str):
        date = pd.to_datetime(date)

    exists, old_date = check_limits_exist(trade_id, strategy_name, signal_id=signal_id)

    ## Convert to float or None
    delta_limit = float(delta_limit) if not pd.isna(delta_limit) else None
    gamma_limit = float(gamma_limit) if not pd.isna(gamma_limit) else None
    vega_limit = float(vega_limit) if not pd.isna(vega_limit) else None
    theta_limit = float(theta_limit) if not pd.isna(theta_limit) else None
    
    ## If exists, we update
    if exists:
        print(f"Updating existing limits for trade_id={trade_id}, strategy_name={strategy_name}, signal_id={signal_id}, date={old_date} to new date={date} with values: delta={delta_limit}, gamma={gamma_limit}, vega={vega_limit}, theta={theta_limit}")
        ## Delta row
        update_greeks_case(
            trade_id=trade_id,
            strategy_name=strategy_name,
            signal_id=signal_id,
            greeks={
                "delta": delta_limit,
                "gamma": gamma_limit,
                "vega": vega_limit,
                "theta": theta_limit,
                "date": date
            },
            date_=old_date
        )
    
    ## If not exists, we create new rows
    else:
        print(f"Creating new limits for trade_id={trade_id}, strategy_name={strategy_name}, signal_id={signal_id}, date={date} with values: delta={delta_limit}, gamma={gamma_limit}, vega={vega_limit}, theta={theta_limit}")
        save_limits_by_meta(
            trade_id=trade_id,
            strategy_name=strategy_name,
            signal_id=signal_id,
            greeks={
                "delta": delta_limit,
                "gamma": gamma_limit,
                "vega": vega_limit,
                "theta": theta_limit
            },
            date_=date
        )
        

def check_limits_exist(trade_id, strategy_name, signal_id):
    """
    Check if the limits exist for the given trade_id and strategy_name
    
    Args:
        trade_id (str): The trade ID to check.
        strategy_name (str): The strategy name to check.
        signal_id (str): The signal ID to check.
    Returns:
        bool: True if limits exist, False otherwise.
    """

    limits_data = db.query_database(
        db='portfolio_data',
        table_name='limits',
        query=f"""
            SELECT * FROM limits
            WHERE trade_id = '{trade_id}'
            AND strategy_name = '{strategy_name}'
            AND signal_id = '{signal_id}'
        """
    )
    if len(limits_data) > 0:
        return True, limits_data["date"].values[0]
    return False, None

def update_limits(trade_id:str, 
                  strategy_name:str, 
                  signal_id:str, 
                  greek_name:str, 
                  update_value:float):
    """
    Update the limits for the given trade_id, strategy_name, signal_id, and greek_name.
    Args:
        trade_id (str): The trade ID to update.
        strategy_name (str): The strategy name to update.
        signal_id (str): The signal ID to update.
        greek_name (str): The greek name to update (e.g., 'delta_limit').
        update_value (float): The new value for the limit.
    """

    dynamic_batch_update(
        db='portfolio_data',
        table_name='limits',
        update_values={"value": update_value},
        condition={
            "trade_id": trade_id,
            "strategy_name": strategy_name,
            "signal_id": signal_id,
            "risk_measure": greek_name
        }
    )

# check_limits_exist(signal_id='AAPL20250821LONG', strategy_name='long_bbands', trade_id='&L:AAPL20260417C265&S:AAPL20260417C270')

In [16]:
## Test limit save
def save_limits_from_backtester(bkt: OptionSignalBacktest, date: datetime = None):
    """
    Save limits from backtester to database.
    If date is provided, only save limits for that date.
    """
    unadjusted_trades = bkt.unadjusted_trades
    for info, greek_meta in bkt.risk_manager.limits_meta.items():
        old_signal_id, trade_id, date_ = info
        signal_id = unadjusted_trades[unadjusted_trades["signal_id"] == old_signal_id]["PT_BKTEST_SIG_ID"].values[0]
        if date is not None:
            if pd.to_datetime(date_).date() != date.date():
                print(f"Skipping {info} as date does not match {date.date()}")
                continue
        store_position_limits(
            delta_limit=greek_meta.get('delta', None),
            gamma_limit=greek_meta.get('gamma', None),
            vega_limit=greek_meta.get('vega', None),
            theta_limit=greek_meta.get('theta', None),
            trade_id=trade_id,
            strategy_name='long_bbands',
            date=date_,
            signal_id=signal_id
        )


## Loading saved limits

In [17]:
_LIMITS = None


In [18]:


@dataclass
class LIMITS:
    refresh_date: datetime = field(default_factory=ny_now)
    __data: pd.DataFrame = None

    @property
    def data(self):
        if self.__data is None or (pd.Timestamp.now() - self.refresh_date).seconds > 3600:
            db = DatabaseAdapter()
            self.__data = db.query_database(
                db='portfolio_data',
                table_name='limits',
                query="""
                    SELECT * FROM limits
                """
            )
            self.refresh_date = pd.Timestamp.now()
            self.__data['value'] = self.__data['value'].astype(float)
        return self.__data

def get_limits_data():
    global _LIMITS
    if _LIMITS is None:
        _LIMITS = LIMITS()
    return _LIMITS.data
_ = get_limits_data()

In [19]:
_LIMITS.data

,date,signal_id,strategy_name,trade_id,risk_measure,value
0,2025-08-21,AAPL20250808LONG,long_bbands,&L:AAPL20260417C265&S:AAPL20260417C270,delta,0.103044
1,2025-08-21,AAPL20250808LONG,long_bbands,&L:AAPL20260417C265&S:AAPL20260417C270,gamma,NaN
2,2025-08-21,AAPL20250808LONG,long_bbands,&L:AAPL20260417C265&S:AAPL20260417C270,theta,NaN
3,2025-08-21,AAPL20250808LONG,long_bbands,&L:AAPL20260417C265&S:AAPL20260417C270,vega,NaN
4,2025-08-29,AMD20250701LONG,long_bbands,&L:AMD20260417C195&S:AMD20260417C200,delta,0.165999
5,2025-08-29,AMD20250701LONG,long_bbands,&L:AMD20260417C195&S:AMD20260417C200,gamma,NaN
6,2025-08-29,AMD20250701LONG,long_bbands,&L:AMD20260417C195&S:AMD20260417C200,theta,NaN
7,2025-08-29,AMD20250701LONG,long_bbands,&L:AMD20260417C195&S:AMD20260417C200,vega,NaN
8,2025-08-25,AMZN20250701LONG,long_bbands,&L:AMZN20260515C280&S:AMZN20260515C285,delta,0.146286
9,2025-08-25,AMZN20250701LONG,long_bbands,&L:AMZN20260515C280&S:AMZN20260515C285,gamma,NaN


In [20]:
def get_position_limit(trade_id:str,
                       strategy_name:str,
                       signal_id:str,
                       risk_measure:str):
    df = get_limits_data()
    assert risk_measure in MEASURES, f"risk_measure must be one of {MEASURES}"
    row = df[
        (df['trade_id'] == trade_id) &
        (df['strategy_name'] == strategy_name) &
        (df['signal_id'] == signal_id) &
        (df['risk_measure'] == risk_measure)
    ]
    if len(row) == 0:
        print(f"No limit found for trade_id={trade_id}, strategy_name={strategy_name}, signal_id={signal_id}, risk_measure={risk_measure}")
        return None
    return float(row['value'].values[0])

get_position_limit(
    trade_id='&L:AAPL20260417C265&S:AAPL20260417C270',
    strategy_name='long_bbands',
    signal_id='AAPL20250808LONG',
    risk_measure='delta'
)

0.103044

In [21]:
portfolio = load_positions_data()[0]

2025-09-17 20:38:56 trade.asset.Stock ERROR: Error getting dividends history for NFLX from yfinance
2025-09-17 20:38:57 trade.asset.Stock ERROR: Probably due to no dividends history


In [22]:
@dataclass
class StrategyLimitsEnabled:
    delta: bool
    gamma: bool
    vega: bool
    theta: bool
    dte: bool
    moneyness: bool
    exercise: bool

def load_config_strategy_limits(strategy_slug: str) -> StrategyLimitsEnabled:
    active_limits = load_config(strategy_slug)['option_settings']['rm_settings']['limits_enabled']
    limits = StrategyLimitsEnabled(
        delta='delta' in active_limits,
        gamma='gamma' in active_limits,
        vega='vega' in active_limits,
        theta='theta' in active_limits,
        dte='dte' in active_limits,
        moneyness='moneyness' in active_limits,
        exercise='exercise' in active_limits,
    )
    return limits

load_config('long_bbands')['option_settings']['rm_settings']
load_config_strategy_limits('long_bbands')

StrategyLimitsEnabled(delta=True, gamma=False, vega=False, theta=False, dte=True, moneyness=True, exercise=False)

In [23]:
sample_position=portfolio.strategies['long_bbands'].positions[0].position_data
sample_active=portfolio.strategies['long_bbands'].positions[0]
sample_option=sample_position.option_data['L'][0]
sample_option.option_type

'C'

In [25]:
from dataclasses import dataclass, field
from datetime import datetime
@dataclass(frozen=True)
class RiskAction:
    date: str|datetime
    action: RMAction
    trade_id: str
    strategy_slug: str

    def __repr__(self):
        return f"RiskAction(date={self.date}, action={self.action}, trade_id={self.trade_id}, strategy_slug={self.strategy_slug})"

@dataclass
class StrategyActions:
    date: str|datetime
    strategy_slug: str
    actions: list[RiskAction] = field(default_factory=list)

    def add_action(self, action: RiskAction):
        self.actions.append(action)

    def __repr__(self):
        actionable = len([a for a in self.actions if type(a.action) != HOLD])
        return f"StrategyActions(date={self.date}, strategy_slug={self.strategy_slug}, new_actions={actionable})"

@dataclass
class PortfolioActions:
    date: str|datetime
    strategy_actions: dict[str, StrategyActions] = field(default_factory=dict)

    def add_strategy_action(self, strategy_action: StrategyActions):
        self.strategy_actions[strategy_action.strategy_slug] = strategy_action

    def __repr__(self):
        return f"PortfolioActions(date={self.date}, strategies={list(self.strategy_actions.keys())})"
    
@dataclass
class PositionLimits:
    """
    Dataclass to hold position limits information.
    """
    delta: float = None
    gamma: float = None
    vega: float = None  
    theta: float = None
    dte: int = None
    moneyness: float = None
    exercise: bool = False
    


In [34]:
def add_position_limits(position: ActivePosition) -> ActivePosition:

    for measure in MEASURES:
        limit = get_position_limit(
            trade_id=position.trade_id,
            strategy_name=position.strategy_name,
            signal_id=position.signal_id,
            risk_measure=measure
        )
        setattr(position.limits, measure, limit)

    config = load_config(position.strategy_name)
    max_moneyness = config['option_settings']['rm_settings']['max_moneyness']
    dte_limit = config['option_settings']['portfolio_settings']['roll_map']
    t_plus_n = config['option_settings']['portfolio_settings']['t_plus_n']
    active_limits = config['option_settings']['rm_settings']['limits_enabled']

    ## DTE limit
    position.limits.dte = dte_limit

    ## Moneyness limit
    moneyness_limit = load_config(position.strategy_name)['option_settings']['rm_settings']['max_moneyness']
    position.limits.moneyness = moneyness_limit

    ## Exercise limit
    position.limits.exercise = 'exercise' in active_limits

    return position
    
add_position_limits(sample_active).limits

PositionLimits(delta=0.103044, gamma=nan, vega=nan, theta=nan, dte=120, moneyness=1.15, exercise=False)

## Analyzing position

In [ ]:
def dte_check(dte: int,
              dte_threshold: int) -> bool:
    """Check if the position's days to expiration (DTE) is below a specified threshold.
    Args:
        dte (int): The days to expiration of the position.
        dte_threshold (int): The threshold value for DTE.
    Returns:
        bool: True if the position's DTE is below the threshold, False otherwise.
    """
    return dte < dte_threshold

def exercise_check(dte: int,
                   t_plus_n: int) -> bool:
    """Check if the position is within a specified number of days to expiration (DTE).
    Args:
        position (EODPositionData): The position data containing DTE information.
        t_plus_n (int): The number of days to expiration threshold.
    Returns:
        bool: True if the position's DTE is less than or equal to t_plus_n, False otherwise.
    """ 
    dte = dte - t_plus_n
    return dte <= 0

def limits_check(position: EODPositionData,
                 limit_lib: dict) -> bool:
    """
    Check if the position's price is within specified limits.
    Args:
        position (EODPositionData): The position data containing price information.
        limit_lib (dict): A dictionary containing 'min_price' and 'max_price' keys.
    Returns:
        bool: True if the position's price is within the limits, False otherwise.
    """
    pass

def moneyness_check(
        moneyness: list,
        moneyness_threshold: float) -> bool:
    """Check if the position's moneyness is below a specified threshold.
    Args:
        moneyness (list): A list of moneyness values for the options in the position.
        moneyness_threshold (float): The threshold value for moneyness.
    Returns:
        bool: True if the position's moneyness is below the threshold, False otherwise.
    """


    return any(abs(m) > abs(moneyness_threshold) for m in moneyness)

def greek_check(
        greek_value: float,
        greek_threshold: float,
        qty: int = 1,
        greater_than: bool = True) -> bool:
    """Check if the position's Greek value is above or below a specified threshold.
    Args:
        greek_value (float): The Greek value of the position (e.g., delta, gamma, vega, theta).
        greek_threshold (float): The threshold value for the Greek.
        greater_than (bool): If True, check if the absolute Greek value is greater than the threshold.
                             else, check if it is less than the threshold.
                             True is for upper limits, False is for lower limits.
    Returns:
        bool: True if the condition is met, False otherwise.
    """
    if greater_than:
        per_greek = greek_value / qty
        _bool = abs(greek_value) > abs(greek_threshold)
        required_qty = max(int(abs(greek_threshold) // abs(per_greek)), 1)
        quantity_diff = abs(qty) - abs(required_qty)
        return _bool, quantity_diff
    else:
        return abs(greek_value) < abs(greek_threshold)

# moneyness_check(sample_position, 0.1)
# dte_check(sample_position, 10)

In [27]:
## Analyze Position
def analyze_position(
    dte: int,
    trade_id: str,
    position_greek_limit: dict,
    dte_limit: int, 
    moneyness_limit: float,
    greeks: dict,
    qty: int,
    moneyness_list: list,
    strategy_enabled_actions: StrategyLimitsEnabled,
    t_plus_n: int,
) -> None:
    """
    Analyzing position data for a given date.
    """
    position_actions = []
    ## DTE Check
    if strategy_enabled_actions.dte:
        if dte_check(dte, dte_limit):
            action = ROLL(trade_id=trade_id)
            action.reason = "not enough DTE"
            position_actions.append(action)

    ## Exercise Check
    if strategy_enabled_actions.exercise:
        if exercise_check(dte, t_plus_n):
            action = EXERCISE(trade_id=trade_id, action=dict())
            action.reason = "position is expiring"
            position_actions.append(action)
    
    ## Moneyness Check
    if strategy_enabled_actions.moneyness:
        if moneyness_check(moneyness_list, moneyness_limit):
            action = ROLL(trade_id=trade_id, action=dict())
            action.reason = "position is too ITM"
            position_actions.append(action)

    ## Greek Checks
    ## Loop through all greek measures
    for greek in MEASURES:

        ## Skip if not enabled
        if strategy_enabled_actions.__dict__[greek]:

            ## Skip if no limit set
            if greek not in position_greek_limit:
                continue
            
            ## Run greek check if limit is set
            _greek_bool, q_diff = greek_check(greek_threshold=position_greek_limit[greek], 
                                              greek_value=greeks[greek], 
                                              greater_than=True,
                                              qty=qty)
            
            ## Positive q_diff if qty is negative (short position is reduced by buying)
            ## Negative q_diff if qty is positive (long position is reduced by selling)
            q_diff = abs(q_diff) if qty < 0 else -abs(q_diff)
            if _greek_bool: ## Only upper limits for now
                action = ADJUST(trade_id=trade_id, action=dict(quantity_diff=q_diff))
                action.reason = f"position {greek} exceeds limit"
                position_actions.append(action)

    ## Finalize action
    ## If no actions, HOLD
    if not position_actions:
        action = HOLD(trade_id=trade_id)
        action.reason = "position within limits"
        return action
    
    ## Else prioritize actions
    ## Prioritize actions:
    ## P1. EXERCISE
    ## P2. ROLL
    ## P3. ADJUST
    else:
        action_priority = {EXERCISE: 1, ROLL: 2, ADJUST: 3, HOLD: 4}
        position_actions = sorted(position_actions, key=lambda x: action_priority[type(x)])
        
        ## If multiple adjust actions, keep only one with max abs quantity diff
        if type(position_actions[0]) in [EXERCISE, ROLL]:
            return position_actions[0]
        elif type(position_actions[0]) == ADJUST:
            adjust_actions = [act for act in position_actions if type(act) == ADJUST]
            if len(adjust_actions) > 1:
                adjust_actions = sorted(adjust_actions, key=lambda x: abs(x.action['quantity_diff']), reverse=True)
            return adjust_actions[0]
        


In [28]:

def analyze_portfolio_positions(portfolio: PortfolioPositions) -> dict:
    """
    Analyzing portfolio positions for a given date.
    """
    portfolio_actions = PortfolioActions(date=portfolio.date)
    for strategy_slug, strategy in portfolio.strategies.items():
        strategy_limits = load_config_strategy_limits(strategy_slug)
        strategy_action = StrategyActions(date=portfolio.date, strategy_slug=strategy_slug)
        for position in strategy.positions:
            ## Calculate moneyness for all options in the position
            options = [
                x for opt_list in position.position_data.option_data.values() for x in opt_list
            ]
            moneyness = []

            for opt in options:
                spot = opt.stock_data.last_price
                moneyness.append(float(opt.strike/spot if opt.option_type.upper() == 'P' else spot/opt.strike))
            position_greek_limits = dict()
            
            ## Get position limits from database
            for greek in MEASURES:
                limit = get_position_limit(
                    trade_id=position.trade_id,
                    strategy_name=strategy_slug,
                    signal_id=position.signal_id,
                    risk_measure=greek
                )

                ## If limit is None: Alert and continue
                if limit is None:
                    print(f"No limit found for trade_id={position.trade_id}, strategy_name={strategy_slug}, signal_id={position.signal_id}, risk_measure={greek}. Using infinity as limit.")
                    continue
                ## If limit is NaN: Alert and continue
                elif pd.isna(limit) or limit is None or np.isinf(limit):
                    print(f"Limit is NaN for trade_id={position.trade_id}, strategy_name={strategy_slug}, signal_id={position.signal_id}, risk_measure={greek}. Using infinity as limit.")
                    continue
                ## 
                position_greek_limits[greek] = limit
            
            ## Get greeks from position
            greeks = dict(
                delta=position.position_data.greeks.binomial_delta,
                gamma=position.position_data.greeks.binomial_gamma,
                vega=position.position_data.greeks.binomial_vega,
                theta=position.position_data.greeks.binomial_theta
            )
            
            ## Get other limits from config
            max_moneyness = load_config(strategy_slug)['option_settings']['rm_settings']['max_moneyness']
            dte_limit = load_config(strategy_slug)['option_settings']['portfolio_settings']['roll_map']
            t_plus_n = load_config(strategy_slug)['option_settings']['portfolio_settings']['t_plus_n']

            ## Analyze position
            action = analyze_position(
                dte=position.position_data.dte,
                trade_id=position.trade_id,
                position_greek_limit=position_greek_limits,
                dte_limit=dte_limit,
                moneyness_limit=max_moneyness,
                greeks=greeks,
                qty=position.quantity,
                strategy_enabled_actions=strategy_limits,
                moneyness_list=moneyness,
                t_plus_n=t_plus_n,
            )
            strategy_action.add_action(RiskAction(date=portfolio.date, 
                                                  action=action, 
                                                  trade_id=position.trade_id, 
                                                  strategy_slug=strategy_slug))
        
            print(f"Trade ID: {position.trade_id}, Action: {type(action).__name__}, Reason: {action.reason}, Details: {getattr(action, 'action', None)}")
        portfolio_actions.add_strategy_action(strategy_action)
            # Store or process the action as needed
    return portfolio_actions
            
portfolio_actions = analyze_portfolio_positions(portfolio)


Limit is NaN for trade_id=&L:AAPL20260417C265&S:AAPL20260417C270, strategy_name=long_bbands, signal_id=AAPL20250808LONG, risk_measure=gamma. Using infinity as limit.
Limit is NaN for trade_id=&L:AAPL20260417C265&S:AAPL20260417C270, strategy_name=long_bbands, signal_id=AAPL20250808LONG, risk_measure=vega. Using infinity as limit.
Limit is NaN for trade_id=&L:AAPL20260417C265&S:AAPL20260417C270, strategy_name=long_bbands, signal_id=AAPL20250808LONG, risk_measure=theta. Using infinity as limit.
Trade ID: &L:AAPL20260417C265&S:AAPL20260417C270, Action: ADJUST, Reason: position delta exceeds limit, Details: {'quantity_diff': -1}
Limit is NaN for trade_id=&L:AMD20260417C195&S:AMD20260417C200, strategy_name=long_bbands, signal_id=AMD20250701LONG, risk_measure=gamma. Using infinity as limit.
Limit is NaN for trade_id=&L:AMD20260417C195&S:AMD20260417C200, strategy_name=long_bbands, signal_id=AMD20250701LONG, risk_measure=vega. Using infinity as limit.
Limit is NaN for trade_id=&L:AMD20260417C19

## To-Do:

- Store limits
- Create return dataset for actions
- Create limits data class, add to load_positions_data


## Tomorrow:
- Move limits to py file
- Add limits dataclass to load_positions_data
- bool to load_scenarios data.
- gradually breakdown create order
- Add StrategyLimitsEnabled to Strategy class

In [31]:
# StrategyLimitsEnabled()
strategy_slug = 'long_bbands'
load_config(strategy_slug)['option_settings']['portfolio_settings']['roll_map']

120